# Ce notebook est *en cours de rédaction*
- Je vais implémenter une fonction, en [Python 3](https://docs.python.org/3/), qui permettra de résoudre rapidement un problème mathématique.

---
## Exposé du problème :
- Soit $n \geq 1$ un nombre de faces pour des dés bien équilibrés. On prendre $n = 6$ pour commencer, mais $n = 8, 10, 12, 20$ ou $n = 100$ est aussi possible.
- On veut trouver deux ensembles d'entiers, $A$ et $B$, de tailles $n$, tels que les entiers dans $A$ soient tous [premiers](https://fr.wikipedia.org/wiki/Nombre_premier), et les entiers dans $B$ soient tous [pairs](https://fr.wikipedia.org/wiki/Parit%C3%A9_(arithm%C3%A9tique)), et on souhaite que chaque somme $a + b$ pour $a \in A$ et $b \in B$ soit encore un nombre premier.

Par exemples :
- Avec $n = 1$, prendre $P = \{3\}$ et $B = \{2\}$ fonctionne, comme $3 + 2 = 5$ est premier.
- Avec $n = 2$, prendre $P = \{3, 5\}$ et $B = \{2, 8\}$ fonctionne, comme $3 + 2 = 5$, $3 + 8 = 11$, $5 + 2 = 7$, et $5 + 8 = 13$ sont premiers.
- Avec $n = 6$, c'est tout de suite moins facile à trouver de tête...

### Buts
Notre but est d'abord de trouver vérifier qu'une paire $(A, B)$ donnée soit valide, puis de trouver une paire valide de taille $n$ donnée.
On ne cherchera pas à avoir un algorithme très malin, une énumération exhaustive nous suffira.


Ensuite on cherchera à répondre à une question plus combinatoire : peut-on trouver, à $n$ fixer, la paire valide $(A, B)$ de somme minimale (ou minimisant un certain critère) ?
La réponse sera oui, et pas trop dure à obtenir.

---
## Présentation de ma solution numérique
- Je vais utiliser le module python [`sympy`](http://sympy.org/) et en particulier ses fonctions du module [`sympy.ntheory`](http://docs.sympy.org/latest/modules/ntheory.html) pour avoir facilement accès à une liste de nombres premiers. En particulier, [`primerange`](http://docs.sympy.org/latest/modules/ntheory.html#sympy.ntheory.generate.primerange) sera bien pratique.
- Je vais procéder par énumération totale avec un "doubling trick" : on cherchera toutes les paires $(A, B)$ bornées dans, disons, $[1, 100]$, puis $[1, 110]$ et ainsi de suite (avec un petit incrément), jusqu'à en trouver une qui marche.
- Cette approche "ascendante" garantit de terminer, pour peu qu'on puisse prouver théoriquement l'existence de la solution qu'on cherche.

### Solution théorique ?
> Je ne vais pas rentrer dans les détails, mais avec [le théorème de la progression arithmétique de Dirichlet](https://fr.wikipedia.org/wiki/Théorème_de_la_progression_arithmétique) (cf. aussi [ce document](http://perso.eleves.ens-rennes.fr/~ariffaut/Agregation/Dirichlet.pdf)), on peut montrer que pour tout nombre de faces $n \geq 1$, on peut trouver un nombre infini d'ensembles $(A, B)$ qui conviennent.
> *Ouais, c'est balèze*.

----
### Solution numérique
On commence avec les dépendances :

In [29]:
from sympy import isprime
from sympy import primerange

---
## Implémentation des fonctions utilitaires requises
D'abord, une fonction `verifie_paire` qui vérifie si une paire $(A, B)$, donnée sous forme de deux *itérateurs* (liste ou ensemble, peu importe), est valide.

In [53]:
def verifie_paire(A, B):
    """Version efficace, qui s'arrête le plus tôt possible."""
    for a in A:
        # if not isprime(a):  # Inutile si A a été bien choisie
        #    return False
        for b in B:
            if not isprime(a + b):
                return False
    return True

Pour visualiser un peu, on en fait une version qui parle :

In [31]:
def verifie_paire_debug(A, B):
    """Version plus lente qui fait tous les tests, et affiche vrai ou faux pour chaque somme a + b."""
    reponse = True
    for a in A:
        if not isprime(a):
            print("  - a = {:<6} est pas premier, ECHEC ...".format(a))
            reponse = False
        for b in B:
            if not isprime(a + b):
                print("  - a = {:<6} + b = {:<6} donnent {:<6} qui n'est pas premier, ECHEC ...".format(a, b, a + b))
                reponse = False
            else:
                print("  - a = {:<6} + b = {:<6} donnent {:<6} qui est bien premier, OK ...".format(a, b, a + b))
    return reponse

Premier exemple :

In [3]:
A = [3]
B = [2]
verifie_paire_debug(A, B)

  - a = 3      + b = 2      donnent 5      qui est bien premier, OK ...


True

Second exemple :

In [5]:
A = (3, 5)
B = (2, 8)
verifie_paire_debug(A, B)

  - a = 3      + b = 2      donnent 5      qui est bien premier, OK ...
  - a = 3      + b = 8      donnent 11     qui est bien premier, OK ...
  - a = 5      + b = 2      donnent 7      qui est bien premier, OK ...
  - a = 5      + b = 8      donnent 13     qui est bien premier, OK ...


True

In [6]:
verifie_paire(A, B)

True

---
On a besoin de cette fonction de combinaison :

In [32]:
from itertools import combinations

Ensuite, on écrit une fonction qui va énumérer *tous* les ensembles $(A, B)$ possibles et valides, avec $\max (A \cup B ) \leq M$ pour $M$ fixé.
J'ajoute une borne inférieure $m$, valant par défaut $m = 1$, pour rester le plus générique possible, mais on ne s'en sert pas vraiment.

- On récupère les candidats possibles pour $a \in A$ via `primerange` (notés $C_A$), et pour $B$ via un `range` à pas $2$ pour considérer seulement des nombres pairs (notés $B_A$),
- Ensuite on boucle sur tous les ensembles $A$ de taille $n$ dans $C_A$, et $B$ de taille $n$ dans $C_B$, via [`itertools.combinations`](https://docs.python.org/3/library/itertools.html#itertools.combinations),
- On garde uniquement ceux qui sont valides, et on les stocke tous.

> **Attention** ça devient vite très couteux !

In [33]:
def enumere_toutes_les_paires(n=2, M=10, m=1, debug=False):
    # Premiers entre m et M (compris), ie. P inter [|m, ..., M|]
    C_A = list(primerange(m, M + 1))
    # Nombres pairs
    if m % 2 == 0:
        C_B = list(range(m, M - 1, 2))
    else:
        C_B = list(range(m + 1, M - 1, 2))
    if debug:
        print("C_A =", C_A)
        print("C_B =", C_B)
        print("Combinaison de n =", n, "éléments dans A =", list(combinations(C_A, n)))
        print("Combinaison de n =", n, "éléments dans B =", list(combinations(C_B, n)))
    # C_A, C_B est déjà trié, c'est cool
    all_A_B = []
    for A in combinations(C_A, n):
        if debug: print(" - A =", A)
        for B in combinations(C_B, n):
            if debug: print("     - B =", B)
            if verifie_paire(A, B):
                if debug: print("==> Une paire (A, B) de plus !")
                all_A_B.append((A, B))
    # all_A_B est aussi trié par ordre lexicographique
    return all_A_B

On peut vérifier que les exemples donnés ci-dessus sont valides, et sont en fait les plus petits :

In [34]:
n = 1
M = 10
enumere_toutes_les_paires(n, M, debug=True)

C_A = [2, 3, 5, 7]
C_B = [2, 4, 6, 8]
Combinaison de n = 1 éléments dans A = [(2,), (3,), (5,), (7,)]
Combinaison de n = 1 éléments dans B = [(2,), (4,), (6,), (8,)]
 - A = (2,)
     - B = (2,)
     - B = (4,)
     - B = (6,)
     - B = (8,)
 - A = (3,)
     - B = (2,)
==> Une paire (A, B) de plus !
     - B = (4,)
==> Une paire (A, B) de plus !
     - B = (6,)
     - B = (8,)
==> Une paire (A, B) de plus !
 - A = (5,)
     - B = (2,)
==> Une paire (A, B) de plus !
     - B = (4,)
     - B = (6,)
==> Une paire (A, B) de plus !
     - B = (8,)
==> Une paire (A, B) de plus !
 - A = (7,)
     - B = (2,)
     - B = (4,)
==> Une paire (A, B) de plus !
     - B = (6,)
==> Une paire (A, B) de plus !
     - B = (8,)


[((3,), (2,)),
 ((3,), (4,)),
 ((3,), (8,)),
 ((5,), (2,)),
 ((5,), (6,)),
 ((5,), (8,)),
 ((7,), (4,)),
 ((7,), (6,))]

In [35]:
n = 2
M = 10
enumere_toutes_les_paires(n, M, debug=True)

C_A = [2, 3, 5, 7]
C_B = [2, 4, 6, 8]
Combinaison de n = 2 éléments dans A = [(2, 3), (2, 5), (2, 7), (3, 5), (3, 7), (5, 7)]
Combinaison de n = 2 éléments dans B = [(2, 4), (2, 6), (2, 8), (4, 6), (4, 8), (6, 8)]
 - A = (2, 3)
     - B = (2, 4)
     - B = (2, 6)
     - B = (2, 8)
     - B = (4, 6)
     - B = (4, 8)
     - B = (6, 8)
 - A = (2, 5)
     - B = (2, 4)
     - B = (2, 6)
     - B = (2, 8)
     - B = (4, 6)
     - B = (4, 8)
     - B = (6, 8)
 - A = (2, 7)
     - B = (2, 4)
     - B = (2, 6)
     - B = (2, 8)
     - B = (4, 6)
     - B = (4, 8)
     - B = (6, 8)
 - A = (3, 5)
     - B = (2, 4)
     - B = (2, 6)
     - B = (2, 8)
==> Une paire (A, B) de plus !
     - B = (4, 6)
     - B = (4, 8)
     - B = (6, 8)
 - A = (3, 7)
     - B = (2, 4)
     - B = (2, 6)
     - B = (2, 8)
     - B = (4, 6)
     - B = (4, 8)
     - B = (6, 8)
 - A = (5, 7)
     - B = (2, 4)
     - B = (2, 6)
     - B = (2, 8)
     - B = (4, 6)
     - B = (4, 8)
     - B = (6, 8)


[((3, 5), (2, 8))]

On peut continuer, avec $n = 3$ et un petit majorant $M$ :

In [36]:
n = 3
M = 20
enumere_toutes_les_paires(n, M, debug=False)

[((3, 7, 13), (4, 10, 16)), ((5, 11, 17), (2, 6, 12))]

On peut continuer, avec $n = 4$ et un petit majorant $M$ :

In [37]:
n = 4
M = 40
enumere_toutes_les_paires(n, M, debug=False)

[((3, 7, 13, 37), (4, 10, 16, 34)),
 ((7, 11, 17, 31), (6, 12, 30, 36)),
 ((7, 13, 19, 37), (4, 10, 24, 34)),
 ((7, 13, 31, 37), (6, 10, 16, 30)),
 ((7, 17, 23, 37), (6, 24, 30, 36))]

On voit que ça commence à prendre du temps. On a pas besoin de toutes les calculer, en fait.

---
## La fonction proposée
- On n'a pas besoin d'énunumérer toutes les paires, il suffit de donner la premier trouvée.
- Pourquoi la première ? Et bien si $C_A$ et $C_B$ sont triés, les candidats pour $(A, B)$ seront aussi triés par ordre lexicographique, et donc le premier trouvé sera le plus petit.

In [43]:
def premiere_paire(n=2, M=10, m=1):
    # Premiers entre m et M (compris), ie. P inter [|m, ..., M|]
    C_A = list(primerange(m, M + 1))
    # Nombres pairs
    if m % 2 == 0:
        C_B = list(range(m, M - 1, 2))
    else:
        C_B = list(range(m + 1, M - 1, 2))
    # C_A, C_B est déjà trié, c'est cool
    for A in combinations(C_A, n):
        for B in combinations(C_B, n):
            if verifie_paire(A, B):
                return (A, B)
    return (None, None)

In [39]:
n = 4
M = 40
A, B = premiere_paire(n, M)
print("A =", A)
print("B =", B)

A = (3, 7, 13, 37)
B = (4, 10, 16, 34)


Avec $n = 5$, on commence à avoir besoin d'aller plus loin que ce majorant `M = 40` :

In [44]:
n = 5
M = 40
A, B = premiere_paire(n, M)  # (None, None) indique qu'on a pas trouvé
print("A =", A)
print("B =", B)

A = None
B = None


Mais parfois on ne sait pas trop quelle valeur donner à ce majorant `M`...

Une approche simple est donc d'augmenter sa valeur jusqu'à trouver une paire valide.

In [48]:
def premiere_paire_explore_M(n=2, Ms=[10,20,30,40,50], m=1):
    for M in Ms:
        resultat = premiere_paire(n=n, M=M, m=m)
        if resultat[0] is not None:
            return resultat
    return (None, None)

On peut faire encore mieux, en augmentant `M` automatiquement d'un certain offset $\delta_M > 0$ :

In [60]:
def premiere_paire_augmente_M(n=2, Mmin=10, deltaM=10, m=1):
    assert isinstance(deltaM, int)
    assert deltaM >= 1
    M = Mmin
    while True:
        print("Appel à premiere_paire(n={}, M={}, m={}) ...".format(n, M, m))
        # Ce n'est pas dangereux, puisqu'on est garanti de trouver une paire qui marche 
        resultat = premiere_paire(n=n, M=M, m=m)
        if resultat[0] is not None:
            print("Terminé, avec M =", M)
            return resultat
        M += deltaM

On peut retrouver le résultat trouvé plus haut, pour $n = 2$ :

In [61]:
n = 2
Ms = [10,20,30,40,50]
A, B = premiere_paire_explore_M(n, Ms)
print("A =", A)
print("B =", B)

A = (3, 5)
B = (2, 8)


In [66]:
n = 4
M = 40
deltaM = 10
A, B = premiere_paire_augmente_M(n, M, deltaM)
print("A =", A)
print("B =", B)

Appel à premiere_paire(n=4, M=40, m=1) ...
Terminé, avec M = 40
A = (3, 7, 13, 37)
B = (4, 10, 16, 34)


Et on peut résoudre pour $n = 5$ et $n = 6$ :

In [ ]:
n = 5
M = 50
deltaM = 50
A, B = premiere_paire_augmente_M(n, M, deltaM)
print("A =", A)
print("B =", B)

Appel à premiere_paire(n=5, M=50, m=1) ...


> *Note :* Plutôt que d'écrire une fonction (qui `return` le premier résultat), on écrit un **générateur**, qui `yield` les résultats un à un.

In [46]:
def premiere_paire_generateur(n=2, M=10, m=1):
    # Premiers entre m et M (compris), ie. P inter [|m, ..., M|]
    C_A = list(primerange(m, M + 1))
    # Nombres pairs
    if m % 2 == 0:
        C_B = list(range(m, M - 1, 2))
    else:
        C_B = list(range(m + 1, M - 1, 2))
    # C_A, C_B est déjà trié, c'est cool
    for A in combinations(C_A, n):
        for B in combinations(C_B, n):
            if verifie_paire(A, B):
                yield (A, B)

---
## Exemples

---
### Exemple avec des dés à 6 faces

In [ ]:
n = 6
M = 400
A, B = premiere_paire(n, M)
print("A =", A)
print("B =", B)

---
### Exemples avec des dés à 8, 10, 12 et 20 faces
> Ça va commencer à être *très couteux*. Je recommande de ne pas faire tourner ce code sur votre machine, mais sur un serveur de calcul, par exemple !

In [ ]:
n = 8
Mmin = 400
deltaM = 50
A, B = premiere_paire_augmente_M(n, M, deltaM)
print("A =", A)
print("B =", B)

In [ ]:
n = 10
Mmin = 400
deltaM = 50
A, B = premiere_paire_augmente_M(n, M, deltaM)
print("A =", A)
print("B =", B)

In [ ]:
n = 12
Mmin = 500
deltaM = 50
A, B = premiere_paire_augmente_M(n, M, deltaM)
print("A =", A)
print("B =", B)

In [ ]:
n = 20
Mmin = 1000
deltaM = 50
A, B = premiere_paire_augmente_M(n, M, deltaM)
print("A =", A)
print("B =", B)

### Exemples avec des dés de tailles différentes